# Topic Modeling and Natural Language Processing with Twitter Data

##  Jason Anastasopoulos
##  February 28, 2018
### Email: ljanastas@princeton.edu

The code below provides a brief introduction on acquiring Twitter data using the twitter API via Python. For this exercise I will be acquiring Donald Trump's tweets and will try to figure out what the topics his tweets are using the Latent Dirichlet Allocation  Topic Model.

In [15]:
import csv
import os,re,csv
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim import corpora, models
import gensim
import numpy as np
import scipy
import math
import matplotlib.pyplot as plt
import plotly.plotly as py
import twitter
import json

Here we enter our Twitter credentials. These can be acquired through 

In [16]:
api = twitter.Api(consumer_key='mkz0izzVKDRzkrR4GoyN9FStT',
                      consumer_secret='4A1YGFEixYmyUNf2idYC33GKCuFoyJkyKpQVXIXCpDedZe0nOt',
                      access_token_key='18249358-xZGyGz8sWmQ9oJ1TBsLKEczwtO24aJ0Q4waDbjxAd',
                      access_token_secret='uqH7cC5BLS65iuAEPEv4TXEtUZvFD80wH03xkqiB7SP7Y')

print(api.VerifyCredentials())

{"created_at": "Fri Dec 19 19:38:51 +0000 2008", "description": "Microsoft Visiting Professor @PrincetonCITP\u25aa\ufe0fAssistant Professor @UGA_PA_Policy & Political Science \u25aa\ufe0f political economy\u25aa\ufe0f machine learning\u25aa\ufe0f causal inference", "favourites_count": 779, "followers_count": 485, "friends_count": 532, "geo_enabled": true, "id": 18249358, "id_str": "18249358", "lang": "en", "listed_count": 33, "location": "Princeton, NJ", "name": "Jason Anastasopoulos", "profile_background_color": "C0DEED", "profile_background_image_url": "http://pbs.twimg.com/profile_background_images/561147677/sproul.jpg", "profile_background_image_url_https": "https://pbs.twimg.com/profile_background_images/561147677/sproul.jpg", "profile_banner_url": "https://pbs.twimg.com/profile_banners/18249358/1510588727", "profile_image_url": "http://pbs.twimg.com/profile_images/953308288044683264/pfRYpPeN_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/95330828804

Search the Twitter API using keywords.

In [17]:
search = api.GetSearch("Oscars") # Replace happy with your search
for tweet in search:
    print(tweet.id, tweet.text)
    
len(search)

(970488599597600769, u'Congratulations @kobebryant!! More hardware for the Mamba. #Oscars https://t.co/3RY30WUYA6')
(970489801576136704, u'Congratulations to the cast &amp; crew #PixarCoco on their Academy Award for Best Animated Feature! #Oscars https://t.co/EmRFd8ccCI')
(970508113928269824, u'Amazing. \n\n#Oscars https://t.co/xoK6HBHM69')
(970866438956253185, u'RT @mayatitos: Qu\xe9 Eiza Gonz\xe1lez en los #Oscars #Oscars90 te gusta m\xe1s https://t.co/8VaoLfrKF9')
(970866438952181760, u'RT @_AlexHirsch: Oh, so when *I* do a story about fish romance its no big deal, but when  @RealGDT does it, it\u2019s Oscars ahoy! Where\u2019s MY st\u2026')
(970866438800945152, u'RT @rdjcoldplay: in memory of jake gyllenhaal\nhe ain\u2019t dead, but the oscars like to pretend that he is #Oscars https://t.co/oUlqVrtsRs')
(970866438578696192, u'RT @RealSteveRohr: #redcarpetready thanks to my new go-to @suitsupply! Impossible not to feel sharp in @suitsupply #tuxedo. Serious threads\u2026')
(970866438

15

The Python twitter library has a lot of cool functions that you can use and learn about through the help() function

In [18]:
help(api.GetUserTimeline)


Help on method GetUserTimeline in module twitter.api:

GetUserTimeline(self, user_id=None, screen_name=None, since_id=None, max_id=None, count=None, include_rts=True, trim_user=False, exclude_replies=False) method of twitter.api.Api instance
    Fetch the sequence of public Status messages for a single user.
    
    The twitter.Api instance must be authenticated if the user is private.
    
    Args:
      user_id (int, optional):
        Specifies the ID of the user for whom to return the
        user_timeline. Helpful for disambiguating when a valid user ID
        is also a valid screen name.
      screen_name (str, optional):
        Specifies the screen name of the user for whom to return the
        user_timeline. Helpful for disambiguating when a valid screen
        name is also a user ID.
      since_id (int, optional):
        Returns results with an ID greater than (that is, more recent
        than) the specified ID. There are limits to the number of
        Tweets which c

In [26]:
t = api.GetUserTimeline(screen_name="realDonaldTrump", count=5000)
tweets = [i.AsDict() for i in t]

tweettext = [i["text"] for i in tweets]

len(tweettext)

200

Loads the text data which are all referenda in New Mexico from 2000-2014. Splits referenda using "####" demarcation, extracts each bill and places it into a list.

## Cleaning the text

In [27]:
######## So far so good not lets clean this up ###
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
texts = []

Here we are pre-processing the text by creating a tokenizer that splits the documents up into tokens (words or phrases), creating a dictionary of stop words and creating a "stemmer" which stems the words (ie removing "-ing" endings etc. We also remove extraneous "bill related" words such as "propXX_XXXX".

In [28]:
for i in tweettext:
    #print "Processing",i
    # clean and tokenize document string
    tokens = tokenizer.tokenize(i)
    # remove all numbers
    tokens = [x for x in tokens if not (x.isdigit() or x[0] == '-' and x[1:].isdigit())]
    # remove structural words
    tokens = [x for x in tokens if len(x) > 1]
    tokens = [x.lower() for x in tokens]
    tokens = [x for x in tokens if 'http' not in x]
    tokens = [x for x in tokens if x not in "_"]
    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    # add tokens to list
    texts.append(stemmed_tokens)

dictionaryall = corpora.Dictionary(texts)

corpusall = [dictionaryall.doc2bow(text) for text in texts]

texts[0]



[u'march',
 u'5th',
 u'democrat',
 u'nowher',
 u'found',
 u'daca',
 u'gave',
 u'month',
 u'just',
 u'don',
 u'care',
 u'co',
 u'vucavygcb7']

In [30]:
tweettext[0]

u'It\u2019s March 5th and the Democrats are nowhere to be found on DACA. Gave them 6 months, they just don\u2019t care. Where a\u2026 https://t.co/VuCAvyGCb7'

This code performs tokenization, stop word removal and number removal and places the corpora into a clean list that will be ready for analysis using the Latent Dirichlet Allocation. 

Notice that there are two sets of texts that are jointly modeled "commercetexts" which are the "business friendly" Chamber of Commerce bills and "nmtexts" which are the New Mexico state referenda.

## Estimating the Latent Dirichlet Allocation model

In [46]:
# generate LDA model
ldamodelall = gensim.models.ldamodel.LdaModel(corpusall, num_topics=5, id2word = dictionaryall, passes=20,
                                              minimum_probability=0)



The code above estimates a 5 topic topic model using Trump's tweets

In [47]:
# First 25 are the Chamber of Commerce Bills Remaining are the propositions so...

print(ldamodelall.print_topics(num_topics=5, num_words=5))

[(0, u'0.032*"co" + 0.012*"just" + 0.009*"nation" + 0.007*"daca" + 0.007*"great"'), (1, u'0.048*"co" + 0.012*"presid" + 0.011*"democrat" + 0.010*"elect" + 0.009*"great"'), (2, u'0.051*"co" + 0.017*"will" + 0.013*"today" + 0.010*"peopl" + 0.007*"republican"'), (3, u'0.051*"co" + 0.007*"rt" + 0.007*"daca" + 0.007*"fake" + 0.006*"news"'), (4, u'0.050*"co" + 0.020*"great" + 0.009*"today" + 0.007*"honor" + 0.007*"state"')]


Prints the first 5 topics from the full model.

This code extracts the topic proportion distributions from the joint Chamber of Commerce and New Mexico referendum model and plots the topic proprtions for one Chamber of Commerce bill (top) and one New Mexico referendum.